<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torchvision import datasets, transforms
import numpy as np

train_mnist = datasets.MNIST("", train=True, transform=transforms.ToTensor(), download=True)

test_mnist = datasets.MNIST("", train=False, transform=transforms.ToTensor(), download=True)

train = []
test = []
for data in train_mnist:
  train.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])

for data in test_mnist:
  test.append([data[0].view(28*28).numpy(), np.eye(10)[data[1]]])


print(len(train))
print(len(test))
print(len(train[0][0]), type(train[0][0]))

60000
10000
784 <class 'numpy.ndarray'>


In [0]:
np.random.seed(1)
num_input = 784
num_output = 10
weight = np.random.random((num_output, num_input))
bias = 0.0

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def forward(x, y):
  x = data[0]
  y = data[1]
  net = [0] * num_output
  for i in range(num_output):
    for j in range(num_input):
      net[i] += (x[j] * weight[i][j] + bias)
  return softmax(net)

# MSE
def calc_loss(y, y_pred):
  #print(y)
  #print(y_pred)
  return ((y-y_pred)**2).mean()

# gradient of MSE dJ/dw = 1/N 2x (w*x - y)
def calc_gradient(x, y, y_pred):
  return (np.dot(2*x, y_pred - y)).mean()

#print("prediction before training : ", forward(10))

learning_rate = 0.01

for epoch in range(15):
  BATCH_SIZE = 20 # without replacement
  for data in train:
    x, y = data[0], data[1]
    y_pred = forward(x, y)
    #print(sum(y_pred))
    loss = calc_loss(y, y_pred)

    grad = calc_gradient(x, y, y_pred)
    break

  w -= learning_rate * grad

  print("epoch : ", epoch, "weight : ", w, "loss : ", loss)


ValueError: ignored